In [ ]:
!pip install fsspec==2024.10.0 --upgrade -q transformers huggingface_hub peft \
  accelerate bitsandbytes datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `LIN371` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LIN371`


In [ ]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"
device = "cuda"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
pip install -U bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_int_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModel.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"": 0})


Unused kwargs: ['load_int_4bit']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_embedding(text):
    # was having device mismatch errors
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()
    return embeddings

# Test example
patient_description = "Chronic fatigue and muscle pain"

trial_descriptions = [
    "Study on fibromyalgia treatment",
    "Research on chronic pain management",
    "Clinical trial for asthma patients"]

patient_embedding = get_embedding(patient_description)
trial_embeddings = [get_embedding(trial) for trial in trial_descriptions]

similarities = [cosine_similarity(patient_embedding, trial_emb)[0][0] for trial_emb in trial_embeddings]

ranked_trials = sorted(zip(trial_descriptions, similarities), key=lambda x: x[1], reverse=True)
for trial, score in ranked_trials:
    print(f"{trial}: {score:.4f}")

Research on chronic pain management: 0.7468
Study on fibromyalgia treatment: 0.6725
Clinical trial for asthma patients: 0.6538


In [ ]:
# Clean and make sense of clinical trial data
import pandas as pd

trial_df = pd.read_csv('ctg-studies.csv')
trial_df.dropna(how='any')
trial_dict = {} # Maps from trial ID to the population it treats

trial_df['Conditions'] = trial_df['Conditions'].fillna('').astype(str).str.lower()
trial_df['Sex'] = trial_df['Sex'].fillna('').astype(str).str.lower()
trial_df['Age'] = trial_df['Age'].fillna('').astype(str).str.lower()

trial_df = trial_df.sample(n=3000, random_state=42)

# Define age and sex categories
age_dict = {"child": (0, 18),
            "child, adult": (0, 65),
            "child, adult, older_adult": (0, 100),
            "adult": (18, 65),
            "adult, older_adult": (18, 100),
            "older_adult": (65, 100)}

sex_dict = {"male": 0,
            "female": 1}

for row in range(len(trial_df)):
  sex = 2

  if trial_df.iloc[row]['Sex'] in sex_dict:
    sex = sex_dict[trial_df.iloc[row]['Sex']]

  age = (0, 100)

  if trial_df.iloc[row]['Age'] in age_dict:
    age = age_dict[trial_df.iloc[row]['Age']]

  curr_trial = {"Condition": trial_df.iloc[row]['Conditions'],
                "Age": age,
                "Sex": sex}

  trial_dict[trial_df.iloc[row]['NCT Number']] = curr_trial

print(trial_dict)

<ipython-input-1-4dfcb11baf45>:4: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  trial_df = pd.read_csv('ctg-studies.csv')


{'NCT06637189': {'Condition': 'stimulation in the ovary|embryo|oocyte|oocyte retrieval|fertilization in vitro|blastocyst|pgt-a', 'Age': (18, 65), 'Sex': 1}, 'NCT06609863': {'Condition': 'advanced hepatocellular carcinoma|atezolizumab|bevacizumab|chemotherapy', 'Age': (18, 100), 'Sex': 2}, 'NCT06684197': {'Condition': 'dexmedetomidine|total intravenous anesthesia|remifentanil consumption|remifentanil|laparoscopic abdominal surgery', 'Age': (18, 100), 'Sex': 2}, 'NCT06593275': {'Condition': 'perception of pain|protection of the palate', 'Age': (18, 100), 'Sex': 2}, 'NCT06647888': {'Condition': 'diabetes mellitus, type 2', 'Age': (18, 100), 'Sex': 2}, 'NCT06609746': {'Condition': 'irritable bowel syndrome (ibs)', 'Age': (18, 100), 'Sex': 2}, 'NCT06699225': {'Condition': 'osteoporosis|pediatric', 'Age': (18, 100), 'Sex': 2}, 'NCT06610071': {'Condition': 'dermatofibrosarcoma protuberans (dfsp)|skin squamous cell carcinoma', 'Age': (0, 100), 'Sex': 2}, 'NCT06634160': {'Condition': 'heterozyg

In [ ]:
print(len(trial_dict.keys()))

# Cache get_embedding for each trial condition
trial_embedding_dict = {}

count = 1

for trial in trial_dict.keys():
    trial_embedding_dict[trial] = get_embedding(trial_dict[trial]['Condition'])
    print(count)
    count += 1

3000
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276

In [ ]:
import pickle

# Save embeddings to a file
# with open("embeddings.pkl", "wb") as f:
#     pickle.dump(trial_embedding_dict, f)

# Load embeddings from the file
with open("embeddings.pkl", "rb") as f:
    loaded_embeddings = pickle.load(f)
print(type(loaded_embeddings))

<class 'dict'>


In [ ]:
# MAKE SURE ALL DATA IS IN THE RIGHT FORMAT
import pandas as pd

patient_df = pd.read_csv('patient_population_data_no_ner.csv')

mrr = 0

def calculate_mrr(patient_condition, ranked_trials):
    normalized_condition = patient_condition.lower().split()

    index = 1
    for trial_id, score, trial_description in ranked_trials:
        # Check if it contains the target condition
        for token in normalized_condition:
          if token in trial_description.lower():
            # Reciprocal rank is 1 divided by the position in the list
            return 1 / index
        index += 1

    # If the condition is not found in any trial, MRR is 0
    return 0.0

# def match_patient_to_trials(patient_df):
trial_similarities = []

for _, patient_row in patient_df.iterrows():
  patient_description = patient_row['clinical_note']
  patient_embedding = get_embedding(patient_description)

  # for the current patient
  patient_trial_similarities = []

  for trial in loaded_embeddings.keys():
      trial_embedding = loaded_embeddings[trial]
      similarity_score = cosine_similarity(patient_embedding, trial_embedding)[0][0]

      age_match = True
      sex_match = True

      # if the patient's age is provided
      if pd.notna(patient_row['AGE']):
          trial_age_range = trial_dict[trial]["Age"]
          if not (trial_age_range[0] <= patient_row['AGE'] <= trial_age_range[1]):
              age_match = False

      # convert patient's sex
      patient_sex = None
      if pd.notna(patient_row['SEX']):
        if patient_row['SEX'].lower() == 'male' or patient_row['SEX'].lower() == 'man':
          patient_sex = 0
        elif patient_row['SEX'].lower() == 'female' or patient_row['SEX'].lower() == 'woman':
          patient_sex = 1
        else:
          patient_sex = 2

      if pd.notna(patient_row['SEX']):
          if trial_dict[trial]["Sex"] != 2 and trial_dict[trial]["Sex"] != patient_sex:
              sex_match = False

      # if both age and sex match (if provided)
      if age_match and sex_match:
          patient_trial_similarities.append((trial, similarity_score, trial_dict[trial]['Condition']))

  # sort
  patient_trial_similarities.sort(key=lambda x: x[1], reverse=True)

  top_trials = patient_trial_similarities[:10]

  actual_patient_description = patient_row['DISEASE_DISORDER']

  mrr += calculate_mrr(actual_patient_description, top_trials)


In [ ]:
print(mrr / len(patient_df))

0.07207190957190956
